In [1]:
import os

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

import cmcrameri.cm as cmc
import math

In [2]:
def myround(x, base=1):
    return base * round(x/base)

In [3]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [4]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):

    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    #print(start, end)
    if start == end:
        end += 1
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [5]:
proportions_list = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

results_db = pd.DataFrame(columns = ['scardec_name', 'magnitude', 'int_magnitude', 'year', 'month', 'day', '10', '20', '30', '40', '50', '60', '70', '80', '90'])

colors = {'10': 'dodgerblue',
          '20': 'darkorange',
          '30': 'green',
          '40': 'red',
          '50': 'purple',
          '60': 'blue',
          '70': 'orange',
          '80': 'green',
          '90': 'red'}


In [6]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_scardec_ye_usgs_sigloch_isc_mag.csv')

In [7]:
combined
combined.columns = ['events', 'scardec', 'ye', 'isc', 'sigloch', 'usgs', 'mag']

In [8]:
catalog = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_m55_catalog.csv', sep = '|')

In [9]:
cols = catalog.columns
column_names = []
for c in cols:
    column_names.append(c.strip().rstrip().lower())
column_names[0] = 'catalog_id'
catalog.columns = column_names

In [10]:
catalog['year'] = catalog.apply(lambda x: x['time'][:4], axis = 1)
catalog['month'] = catalog.apply(lambda x: x['time'][5:7], axis = 1)
catalog['day'] = catalog.apply(lambda x: x['time'][8:10], axis = 1)
catalog['hour'] = catalog.apply(lambda x: x['time'][11:13], axis = 1)
catalog['minute'] = catalog.apply(lambda x: x['time'][14:16], axis = 1)

catalog['event'] = catalog.apply(lambda x: x['year'] + x['month'] + x['day'] + '_' + x['hour'] + x['minute'], axis = 1)

catalog['int_magnitude'] = catalog.apply(lambda x: myround(x['magnitude']), axis = 1)

In [11]:
catalog.drop(columns = ['contributor', 'contributorid', 'magauthor', 'eventlocationname', 'author', 'catalog', 'time'], inplace = True)

In [12]:
catalog = catalog[['event', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'depth/km', 'magnitude', 'int_magnitude', 'magtype']]

In [13]:
for dataset in ['scardec', 'ye', 'usgs', 'isc', 'sigloch']:
	print(f'++++++++++{dataset}++++++++++')
	results_db = pd.read_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_relative_root_times_5.csv')
	results_db.columns = ['event', '5', '10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95']
	results_db.sort_values(by = 'event', inplace = True, ignore_index=True)
	print(results_db)
	results_5 = pd.DataFrame(columns = ['event', '0_5', '5_10', '10_15', '15_20', '20_25', '25_30', '30_35', '35_40', '40_45', '45_50', '50_55', '55_60', '60_65', '65_70', '70_75', '75_80', '80_85', '85_90', '90_95', '95_100'])

	#results_10['event'] = results_db['event']
	if dataset == 'sigloch':
		results_5['event'] = results_db.apply(lambda x: x['event'][:-2], axis = 1)
	else:
		results_5['event'] = results_db['event']

	results_5['0_5'] = results_db['5']
	results_5['5_10'] = results_db['10'] - results_db['5']
	results_5['10_15'] = results_db['15'] - results_db['10']
	results_5['15_20'] = results_db['20'] - results_db['15']
	results_5['20_25'] = results_db['25'] - results_db['20']
	results_5['25_30'] = results_db['30'] - results_db['25']
	results_5['30_35'] = results_db['35'] - results_db['30']
	results_5['35_40'] = results_db['40'] - results_db['35']
	results_5['40_45'] = results_db['45'] - results_db['40']
	results_5['45_50'] = results_db['50'] - results_db['45']
	results_5['50_55'] = results_db['55'] - results_db['50']
	results_5['55_60'] = results_db['60'] - results_db['55']
	results_5['60_65'] = results_db['65'] - results_db['60']
	results_5['65_70'] = results_db['70'] - results_db['65']
	results_5['70_75'] = results_db['75'] - results_db['70']
	results_5['75_80'] = results_db['80'] - results_db['75']
	results_5['80_85'] = results_db['85'] - results_db['80']
	results_5['85_90'] = results_db['90'] - results_db['85']
	results_5['90_95'] = results_db['95'] - results_db['90']
	results_5['95_100'] = 1 - results_db['95']


	results_5_and_catalog = pd.merge(results_5, catalog, on = 'event', how = 'inner')
	results_5_and_catalog = results_5_and_catalog[['event', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 
												  'depth/km', 'magnitude', 'int_magnitude', 'magtype', 
												  '0_5', '5_10', '10_15', '15_20', '20_25', '25_30', '30_35', '35_40', '40_45', '45_50', '50_55', '55_60', '60_65', '65_70', '70_75', '75_80', '80_85', '85_90', '90_95', '95_100']]

	results_5_and_catalog.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_results_5_all.csv', index = False)

	if dataset == 'sigloch':
		results_5_and_catalog_filtered = results_5_and_catalog[results_5_and_catalog['magnitude'] <= 7.2].copy()
	else:
		results_5_and_catalog_filtered = results_5_and_catalog.copy()

	results_5_and_catalog_filtered.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_results_5.csv', index = False)

	print(results_5)


++++++++++scardec++++++++++
              event         5        10        15        20        25  \
0     19920120_1337  0.295775  0.323944  0.352113  0.394366  0.408451   
1     19920213_0129  0.282828  0.348485  0.393939  0.469697  0.565657   
2     19920305_1439  0.250000  0.330000  0.380000  0.420000  0.450000   
3     19920307_0153  0.253968  0.309524  0.341270  0.373016  0.404762   
4     19920313_1601  0.257143  0.295238  0.323810  0.342857  0.361905   
...             ...       ...       ...       ...       ...       ...   
4078  20221208_0050  0.500000  0.547619  0.595238  0.595238  0.642857   
4079  20221211_1431  0.276596  0.351064  0.393617  0.436170  0.457447   
4080  20221214_1840  0.316667  0.383333  0.383333  0.433333  0.433333   
4081  20221220_1034  0.295238  0.423810  0.480952  0.519048  0.595238   
4082  20221228_1634  0.350649  0.415584  0.467532  0.493506  0.532468   

            30        35        40        45        50        55        60  \
0     0.436620  0

In [14]:
results_5

,event,0_5,5_10,10_15,15_20,20_25,25_30,30_35,35_40,40_45,...,50_55,55_60,60_65,65_70,70_75,75_80,80_85,85_90,90_95,95_100
0,19900530_0234,0.189655,0.068966,0.086207,0.172414,0.051724,0.017241,0.017241,0.017241,0.017241,...,0.017241,0.017241,0.000000,0.017241,0.017241,0.034483,0.086207,0.034483,0.068966,6.896552e-02
1,19910903_0905,0.200000,0.100000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,1.000000e-01
2,19920217_0001,0.307692,0.000000,0.000000,0.076923,0.000000,0.153846,0.000000,0.000000,0.000000,...,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.153846,7.692308e-02
3,19920217_0001,0.333333,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,...,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222,1.443290e-15
4,19920302_1229,0.172414,0.103448,0.034483,0.034483,0.000000,0.034483,0.034483,0.000000,0.068966,...,0.000000,0.000000,0.068966,0.034483,0.034483,0.000000,0.068966,0.068966,0.068966,1.379310e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,20180415_1930,0.102564,0.025641,0.000000,0.025641,0.000000,0.051282,0.000000,0.025641,0.051282,...,0.051282,0.051282,0.000000,0.000000,0.000000,0.025641,0.000000,0.051282,0.000000,5.128205e-02
3090,20180419_2109,0.266667,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.133333,0.000000,...,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.133333,0.000000,0.133333,6.666667e-02
3091,20180419_2109,0.200000,0.066667,0.133333,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.066667,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,1.333333e-01
3092,20180429_2223,0.150943,0.056604,0.037736,0.415094,0.075472,0.018868,0.018868,0.018868,0.000000,...,0.000000,0.037736,0.000000,0.000000,0.018868,0.037736,0.000000,0.037736,0.000000,5.660377e-02


In [15]:
results_5_and_catalog

,event,catalog_id,year,month,day,hour,minute,latitude,longitude,depth/km,...,50_55,55_60,60_65,65_70,70_75,75_80,80_85,85_90,90_95,95_100
0,19900530_0234,2814540,1990,05,30,02,34,-6.0307,-77.2142,2.5,...,0.017241,0.017241,0.000000,0.017241,0.017241,0.034483,0.086207,0.034483,0.068966,6.896552e-02
1,19910903_0905,223098,1991,09,03,09,05,-17.8562,-116.0124,8.5,...,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,1.000000e-01
2,19920217_0001,231612,1992,02,17,00,01,79.1748,124.5240,10.0,...,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.153846,7.692308e-02
3,19920217_0001,231612,1992,02,17,00,01,79.1748,124.5240,10.0,...,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222,1.443290e-15
4,19920302_1229,239130,1992,03,02,12,29,52.8835,159.8808,41.7,...,0.000000,0.000000,0.068966,0.034483,0.034483,0.000000,0.068966,0.068966,0.068966,1.379310e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3064,20180415_1930,10728247,2018,04,15,19,30,1.4083,126.8759,34.0,...,0.051282,0.051282,0.000000,0.000000,0.000000,0.025641,0.000000,0.051282,0.000000,5.128205e-02
3065,20180419_2109,10762979,2018,04,19,21,09,-42.7797,42.1877,10.0,...,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.133333,0.000000,0.133333,6.666667e-02
3066,20180419_2109,10762979,2018,04,19,21,09,-42.7797,42.1877,10.0,...,0.066667,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,1.333333e-01
3067,20180429_2223,10766369,2018,04,29,22,23,-39.2647,46.1195,10.0,...,0.000000,0.037736,0.000000,0.000000,0.018868,0.037736,0.000000,0.037736,0.000000,5.660377e-02


In [16]:
results_5_and_catalog_filtered

,event,catalog_id,year,month,day,hour,minute,latitude,longitude,depth/km,...,50_55,55_60,60_65,65_70,70_75,75_80,80_85,85_90,90_95,95_100
0,19900530_0234,2814540,1990,05,30,02,34,-6.0307,-77.2142,2.5,...,0.017241,0.017241,0.000000,0.017241,0.017241,0.034483,0.086207,0.034483,0.068966,6.896552e-02
1,19910903_0905,223098,1991,09,03,09,05,-17.8562,-116.0124,8.5,...,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,1.000000e-01
2,19920217_0001,231612,1992,02,17,00,01,79.1748,124.5240,10.0,...,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.153846,7.692308e-02
3,19920217_0001,231612,1992,02,17,00,01,79.1748,124.5240,10.0,...,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222,1.443290e-15
4,19920302_1229,239130,1992,03,02,12,29,52.8835,159.8808,41.7,...,0.000000,0.000000,0.068966,0.034483,0.034483,0.000000,0.068966,0.068966,0.068966,1.379310e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3064,20180415_1930,10728247,2018,04,15,19,30,1.4083,126.8759,34.0,...,0.051282,0.051282,0.000000,0.000000,0.000000,0.025641,0.000000,0.051282,0.000000,5.128205e-02
3065,20180419_2109,10762979,2018,04,19,21,09,-42.7797,42.1877,10.0,...,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.133333,0.000000,0.133333,6.666667e-02
3066,20180419_2109,10762979,2018,04,19,21,09,-42.7797,42.1877,10.0,...,0.066667,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,1.333333e-01
3067,20180429_2223,10766369,2018,04,29,22,23,-39.2647,46.1195,10.0,...,0.000000,0.037736,0.000000,0.000000,0.018868,0.037736,0.000000,0.037736,0.000000,5.660377e-02


In [17]:
dataset

'sigloch'

In [18]:
for dataset in ['scardec', 'ye', 'usgs', 'isc', 'sigloch']:
	print(f'++++++++++{dataset}++++++++++')
	results_db = pd.read_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_relative_root_times_5.csv')
	results_db.columns = ['event', '5', '10', '15', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95']
	results_db.sort_values(by = 'event', inplace = True, ignore_index=True)
	print(results_db)

++++++++++scardec++++++++++
              event         5        10        15        20        25  \
0     19920120_1337  0.295775  0.323944  0.352113  0.394366  0.408451   
1     19920213_0129  0.282828  0.348485  0.393939  0.469697  0.565657   
2     19920305_1439  0.250000  0.330000  0.380000  0.420000  0.450000   
3     19920307_0153  0.253968  0.309524  0.341270  0.373016  0.404762   
4     19920313_1601  0.257143  0.295238  0.323810  0.342857  0.361905   
...             ...       ...       ...       ...       ...       ...   
4078  20221208_0050  0.500000  0.547619  0.595238  0.595238  0.642857   
4079  20221211_1431  0.276596  0.351064  0.393617  0.436170  0.457447   
4080  20221214_1840  0.316667  0.383333  0.383333  0.433333  0.433333   
4081  20221220_1034  0.295238  0.423810  0.480952  0.519048  0.595238   
4082  20221228_1634  0.350649  0.415584  0.467532  0.493506  0.532468   

            30        35        40        45        50        55        60  \
0     0.436620  0